# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# What test is appropriate for this problem? Does CLT apply?
A hypothesis test comparing population proportions is appropriate for this problem as we want to test against two different population proportions: candidates of white-sounding names with a callback and candidates of black-sounding names with a callback.

The CLT states that the sampling distribution of the sample population proportions approaches a normal distribution as the sample size gets larger, starting with a sample size threshold of at least 30.  Moreover, assuming that each observation in the sample are independent, the CLT does apply to this problem.

# What are the null and alternate hypotheses?
Null hypothesis:  The rate of candidtate interview callbacks for resumes with white-sounding or black-sounding names are the same.

𝐻<sub>0</sub>: p<sub>wc</sub> - p<sub>bc</sub> = 0

Alternate hypothesis:  The rate of candidate interview callbacks for resumes with white sounding names are not the same for those with resumes with black-sounding names.

𝐻<sub>a</sub>: p<sub>wc</sub> - p<sub>bc</sub> ≠ 0

# Compute margin of error, confidence interval, and p-value using both the Bootstrap and Frequentist methods.

In [5]:
# Compute margin of error, confidence interval, and p-value.

# Compute p-value with bootstraps method
w = data[data.race=='w']
b = data[data.race=='b']
wc = np.array(data.loc[data.race=='w','call'])
bc = np.array(data.loc[data.race=='b','call'])

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

# Compute difference of mean impact force from experiment: empirical_diff_means
empirical_diff_means = diff_of_means(wc, bc)

# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(wc, bc, diff_of_means, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)
print('p-value = ' + str(p))

# Calculate bootstrap 95% confidence interval
def draw_bs_reps(data, func, size=1):
    bs_replicates = np.empty(size)
    for i in range(size):
        bs_sample = np.random.choice(data, size=len(data))
        bs_replicates[i] = func(bs_sample)
    return bs_replicates

bs_wc = draw_bs_reps(wc, np.mean, 100000)
bs_bc = draw_bs_reps(bc, np.mean, 100000)
bs_diff_replicates = bs_wc - bs_bc

bs_conf_int = np.percentile(bs_diff_replicates, [0.5, 99.5])
print('95% confidence interval: ' + str(bs_conf_int))

# Compute margin of error
len_wc = len(w.loc[data.call==1])
len_bc = len(b.loc[data.call==1])
prop_wc = len_wc / len(w)
prop_bc = len_bc / len(b)

MoE = 1.96*np.sqrt((prop_wc*(1-prop_wc))/len(w)+(prop_bc*(1-prop_bc))/len(b))
print('Margin of Error = \u00B1' + str(MoE))

print('\nCallback rate for candidates with resumes with white-sounding names = ',prop_wc)
print('Callback rate for candidates with resumes with black-sounding names = ',prop_bc)

p-value = 0.0
95% confidence interval: [ 0.01190965  0.05215606]
Margin of Error = ±0.0152554063499

Callback rate for candidates with resumes with white-sounding names =  0.09650924024640657
Callback rate for candidates with resumes with black-sounding names =  0.06447638603696099


In [6]:
# Compute p-value with frequentist method
phat = ((len(w)*prop_wc)+(len(b)*prop_bc))/(len(w)+len(b))
z_score = diff_of_means(wc, bc) / np.sqrt(phat*(1-phat)*(1/len(w)+1/len(b)))
print('The z-score is '+ str(z_score))

z_pval = stats.norm.sf(np.abs(z_score))*2
print('The p-value is '+ str(z_pval))

freq_conf_int = [diff_of_means(wc, bc)-MoE, diff_of_means(wc, bc)+MoE]
print('95% confidence interval: ' + str(freq_conf_int))

The z-score is 4.10841223524
The p-value is 3.9838854095e-05
95% confidence interval: [0.016777448505174139, 0.047288261204947016]


# Describe the statistical significance in the context or the original problem.
With a significance level of α = 0.05, the resulting p-value of 0 AND with the difference between the two groups (p<sub>wc</sub> - p<sub>bc</sub> = 0) not being convered in our 95% CI allows us to reject the null hypothesis, and conclude the callback rates between resumes with white-sounding and black-sounding names are not the same.

# Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
No, because there could be other resume-related factors from this sample group that could have affected the results and thus could have served as confounding variables in this study.  Other resume-related factors could be levels of experience, education, age, skills, etc.

To control for such possibly confounding factors, I would have sampled this from one age group to ensure ageism and levels of experience did not serve as a advantage or disadvantage against other age groups.  Additionally, I would control for such factors by ensuring each sample met the minimum requirements of education and skills for the the jobs they applied to in order test against only qualified candidates.